In [1]:
%config IPCompleter.evaluation='unsafe'

In [2]:
import sys

sys.path.insert(0, "/Users/neugebauer/gitProjects/pyiron_core")

In [3]:
import pyiron_workflow.graph.graph_operations as go
from pyiron_workflow import as_function_node
from pyiron_workflow.graph import base, gui
from pyiron_workflow.simple_workflow import Workflow
import pyiron_nodes as pn
# from pyironflow.pyironflow import PyironFlow

2025-02-24 18:33:48,472 - pyiron_log - WARNING - pyiron found a 'dynamic' folder in the /Users/neugebauer/pyiron/resources resource directory. These are no longer supported in pyiron_base >=0.7.0. They are replaced by Project.create_job_class() and Project.wrap_python_function().
2025-02-24 18:33:48,473 - pyiron_log - WARNING - pyiron found a 'templates' folder in the /Users/neugebauer/pyiron/resources resource directory. These are no longer supported in pyiron_base >=0.7.0. They are replaced by Project.create_job_class() and Project.wrap_python_function().


In [4]:
import pyiron

In [5]:
# import pyiron_potentialfit.assyst.calculations as pcalc

Why not running the graph rather than the workflow. Graph is much more powerful and includes serialization and subgraphs? This works already! Does not yet have concept of input.

In [6]:
wf = Workflow('test')
wf.bulk = pn.atomistic.structure.build.Bulk('Al', cubic=True)
wf.repeat = pn.atomistic.structure.transform.Repeat(wf.bulk, repeat_scalar=1)
wf.repeat2 = pn.atomistic.structure.transform.Repeat(wf.repeat, repeat_scalar=1)
# wf.lammps = pn.atomistic.engine.lammps.Code1(structure=wf.repeat2)

full_graph = base.get_full_graph_from_wf(wf)
sub_graph = base._get_subgraph(full_graph, [1]) # collaps if node is macro node

# TODO: 
# - extend output option (include all child node outputs)
# - substitute selected nodes with ports having outside connections by a single virtual (graph-only) node
# - add parent group as graph node
# - make all inner nodes to child nodes of the parent group
# - create node object for the parent group

base.GuiGraph(full_graph, full_graph=True, height=400)
# node = base.graph_to_node(sub_graph)
# node
# node.outputs, sub_graph.label, 

ReactFlowWidget(layout=Layout(height='400px', width='800px'))

In [7]:
# TODO: 
# - extend output option (include all child node outputs)
# - substitute selected nodes with ports having outside connections by a single virtual (graph-only) node
# - add parent group as graph node
# - make all inner nodes to child nodes of the parent group
# - create node object for the parent group

In [8]:
full_graph.nodes;

In [9]:
node = base.graph_to_node(sub_graph)
node.inputs

,label,value,type,default,ready,node
0,repeat__structure,label ...,NonPrimitive,NotData,True,<pyiron_workflow.simple_workflow.Node object a...


In [10]:
base.get_externally_connected_input_ports(sub_graph)

[('repeat', 'structure')]

In [11]:
base.get_non_default_input(sub_graph)

{'repeat': {'structure': label                                            structure
  type                                     builtins.NoneType
  ready                                                False
  value                                                 None
  node     <pyiron_workflow.simple_workflow.Node object a...
  dtype: object}}

In [12]:
base.graph_to_node(sub_graph).outputs

,label,value,type,ready,node
0,repeat__structure,None,builtins.str,False,<pyiron_workflow.simple_workflow.Node object a...


In [13]:
new_graph = base.create_group(full_graph, [0,1], label="subgraph")
new_graph.edges

labels ['bulk__name', 'bulk__cubic']
GraphEdge(source='va_i_subgraph__bulk__name', target='bulk', sourceHandle='x', targetHandle='name')
GraphEdge(source='va_i_subgraph__bulk__cubic', target='bulk', sourceHandle='x', targetHandle='cubic')
labels ['repeat__structure']
['subgraph']


,source,target,sourceHandle,targetHandle
0,bulk,repeat,structure,structure
1,va_o_subgraph__repeat__structure,repeat2,x,structure
2,va_i_subgraph__bulk__name,bulk,x,name
3,va_i_subgraph__bulk__cubic,bulk,x,cubic
4,repeat,va_o_subgraph__repeat__structure,structure,x


Remark: Make sure that all operations regarding virtual nodes and edges are encapsulated to not affect any other groups

In [14]:
wf = Workflow("test")
wf.bulk = pn.atomistic.structure.build.Bulk("Al", cubic=True)
wf.repeat = pn.atomistic.structure.transform.Repeat(wf.bulk, repeat_scalar=1)
wf.repeat2 = pn.atomistic.structure.transform.Repeat(wf.repeat, repeat_scalar=1)
wf.lammps = pn.atomistic.engine.lammps.Code1(structure=wf.repeat2)

full_graph = base.get_full_graph_from_wf(wf)

new_graph = base.create_group(full_graph, [0, 1, 2], label="subgraph")

new_graph.nodes["subgraph"].expanded = True
new_graph.nodes["lammps"].expanded = False

# True, True gives error

new_graph = base.get_updated_graph(new_graph)

base.GuiGraph(new_graph, full_graph=True, height=400)

Adding macro node lammps
Expanding node lammps
labels ['bulk__name', 'bulk__cubic']
GraphEdge(source='va_i_subgraph__bulk__name', target='bulk', sourceHandle='x', targetHandle='name')
GraphEdge(source='va_i_subgraph__bulk__cubic', target='bulk', sourceHandle='x', targetHandle='cubic')
labels ['repeat2__structure']
['subgraph']
Expanding node subgraph


ReactFlowWidget(layout=Layout(height='400px', width='800px'))

In [15]:
g = base._remove_virtual_nodes(new_graph, parent_label='lammps')
g = base._remove_virtual_nodes(g, parent_label='subgraph')
g = base.remove_hidden_nodes(g, "lammps")
g.nodes

,id,import_path,label,parent_id,level,node,graph,node_type,widget_type,expanded
0,subgraph,None,subgraph,None,0,<pyiron_workflow.simple_workflow.Node object a...,"Graph(id=None, label='subgraph', root_node=Non...",graph,customNode,True
1,bulk,pyiron_nodes.atomistic.structure.build.Bulk,bulk,subgraph,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
2,repeat,pyiron_nodes.atomistic.structure.transform.Repeat,repeat,subgraph,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
3,repeat2,pyiron_nodes.atomistic.structure.transform.Repeat,repeat2,subgraph,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
4,lammps,pyiron_nodes.atomistic.engine.lammps.Code1,lammps,None,0,<pyiron_workflow.simple_workflow.Node object a...,"Graph(id='lammps', label='lammps', root_node=N...",graph,customNode,False


In [16]:
new_graph.nodes

,id,import_path,label,parent_id,level,node,graph,node_type,widget_type,expanded
0,subgraph,None,subgraph,None,0,<pyiron_workflow.simple_workflow.Node object a...,"Graph(id=None, label='subgraph', root_node=Non...",graph,customNode,True
1,bulk,pyiron_nodes.atomistic.structure.build.Bulk,bulk,subgraph,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
2,repeat,pyiron_nodes.atomistic.structure.transform.Repeat,repeat,subgraph,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
3,repeat2,pyiron_nodes.atomistic.structure.transform.Repeat,repeat2,subgraph,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
4,lammps,pyiron_nodes.atomistic.engine.lammps.Code1,lammps,None,0,<pyiron_workflow.simple_workflow.Node object a...,"Graph(id='lammps', label='lammps', root_node=N...",graph,customNode,False


In [17]:
new_graph.edges

,source,target,sourceHandle,targetHandle
0,bulk,repeat,structure,structure
1,repeat,repeat2,structure,structure
2,bulk,repeat,structure,structure
3,repeat,repeat2,structure,structure
4,repeat2,lammps,structure,structure


In [18]:
new_graph.__getstate__()['nodes']['subgraph']['graph']

{'label': 'subgraph',
 'nodes': {'bulk': {'id': 'bulk',
   'import_path': 'pyiron_nodes.atomistic.structure.build.Bulk',
   'label': 'bulk',
   'parent_id': None,
   'level': 0,
   'graph': None,
   'node_type': 'node',
   'widget_type': 'customNode',
   'expanded': False,
   'node': {'label': 'bulk',
    'function': 'pyiron_nodes.atomistic.structure.build.Bulk',
    'inputs': {'name': 'Al', 'cubic': True}}},
  'repeat': {'id': 'repeat',
   'import_path': 'pyiron_nodes.atomistic.structure.transform.Repeat',
   'label': 'repeat',
   'parent_id': None,
   'level': 0,
   'graph': None,
   'node_type': 'node',
   'widget_type': 'customNode',
   'expanded': False,
   'node': {'label': 'repeat',
    'function': 'pyiron_nodes.atomistic.structure.transform.Repeat',
    'inputs': {}}},
  'repeat2': {'id': 'repeat2',
   'import_path': 'pyiron_nodes.atomistic.structure.transform.Repeat',
   'label': 'repeat2',
   'parent_id': None,
   'level': 0,
   'graph': None,
   'node_type': 'node',
   'widg

In [19]:
new_graph.__getstate__()['nodes']['lammps']['node']['inputs']

{'calculator': 'NotData'}

In [20]:
# base._save_graph(new_graph2, "new_graph2.json")
loaded_graph = base._load_graph("new_graph2.json")

FileNotFoundError: File 'new_graph2.json' not found in dir ..

In [ ]:
base.display_gui_data(new_graph2)


In [ ]:
new_graph = base.create_group(full_graph, [0,1], label="subgraph")
new_graph = base.move_parent_nodes_to_top(new_graph)
new_graph.nodes["subgraph"].expanded = True
new_graph = base.get_updated_graph(new_graph)

# new_graph0 = base.collapse_node(new_graph, "subgraph")
# new_graph = base.expand_node(new_graph, "subgraph")
# new_graph1 = base._remove_virtual_edges(new_graph0)
# new_graph2 = base._remove_edges_to_hidden_nodes(new_graph1)
# new_graph = base._remove_virtual_nodes(new_graph, reconnect_edges=True, parent_label="subgraph")
# new_graph = base._remove_virtual_nodes(new_graph, reconnect_edges=True, parent_label="lammps")
# new_graph = base.update_execution_graph(new_graph)

new_graph.edges

In [ ]:
new_graph0.edges

TODO:
- remove duplicate edges

In [ ]:
xx

In [21]:
from pyiron_workflow.graph import gui

pf = gui.PyironFlow(['flow_macro', 'Code1', 'iterCellSize', 'elastic']) #, 'elastic_constants'])
pf.gui

TODO:
- outlabels to right (.jsx)
- Static as default for Lammps
- energies as list in add energies
- 
- store error for iter when non-json object (fix)
- allow for automatic array notation as input and outputs (like numpy, at least for structure and DataClasses?)
- 

In [22]:
pn.atomistic.assyst.structures.SpaceGroupInput().run()

ValueError: mutable default <class 'list'> for field stoichiometry is not allowed: use default_factory

In [ ]:
from dataclasses import dataclass

@dataclass
class SpaceGroupInput:
    # def __post_init__(self):
    #     if self.stoichiometry is None or len(self.stoichiometry) == 0:
    #         self.stoichiometry = list(range(1, self.max_atoms + 1))
    #     if self.spacegroups is None:
    #         self.spacegroups = list(range(1,231))

    elements: list[str]
    max_atoms: int = 10
    stoichiometry: list[int] | list[tuple[int, ...]] | None = None
    spacegroups: list[int] | None = None

    # can be either a single cutoff distance or a dictionary mapping chemical
    # symbols to min *radii*; you need to half the value if you go from using a
    # float to a dict
    min_dist: float | dict[str, float] | None = None

    # FIXME: just to restrict number of structures during testing
    max_structures: int = 20

SpaceGroupInput()

In [ ]:
bulk = pf.graph.nodes['Bulk'].node
bulk.run()

In [ ]:
wf = Workflow('elastic')
wf.bulk = pn.atomistic.structure.build.CubicBulkCell('Al')
wf.input_elastic_tensor = pn.atomistic.property.elastic.InputElasticTensor()
wf.symmetry_analysis = pn.atomistic.property.elastic.SymmetryAnalysis(structure=wf.bulk, parameters=wf.input_elastic_tensor)
wf.structures = pn.atomistic.property.elastic.GenerateStructures(structure=wf.bulk, analysis=wf.symmetry_analysis, parameters=wf.input_elastic_tensor)
wf.calc = pn.atomistic.engine.lammps.CalcStatic()
#wf.engine = pn.atomistic.engine.lammps.Lammps(structure=wf.bulk, calculator=wf.calc)
# wf.add_energies = pn.atomistic.property.elastic.AddEnergies(structure_container=wf.structures, engine=wf.engine)

wf.run()


In [ ]:
from pyiron_workflow import as_inp_dataclass_node, as_out_dataclass_node
from dataclasses import dataclass, field
import numpy as np


@as_out_dataclass_node
# @dataclass
class OutputElasticSymmetryAnalysis:
    SGN: int = 0
    v0: float = 0.0
    LC: int = 1
    Lag_strain_list: list = field(default_factory=lambda: [])
    epss: np.ndarray = field(default_factory=lambda: np.zeros(0))

out = OutputElasticSymmetryAnalysis()#.dataclass()


In [ ]:
@as_out_dataclass_node
class OutputCalcMD:
    energies_pot: list | np.ndarray = field(
        default_factory=lambda: np.array([])
    )

    energies_kin: list | np.ndarray = field(
        default_factory=lambda: np.array([])
    )
    SGN: int = 0
    v0: float = 0.0
    LC: int = 1    

OutputCalcMD().dataclass()

In [ ]:
pn.atomistic.property.elastic.DataStructureContainer()   # OutputElasticAnalysis()

In [ ]:
pn.atomistic.property.elastic.OutputElasticSymmetryAnalysis() #bulk.run())

In [ ]:
macro_label = 'iterate_1'
macro_node = pf.graph.nodes[macro_label].node
# macro_graph = base.get_graph_from_macro_node(macro_node)

out_list = macro_node.outputs.out_lst.value

def ExtractList(out_list, label, flatten=True):
    import numpy as np
    
    collect = np.array([out.__getattribute__(label) for out in out_list])
    if flatten:
        collect = collect.flatten()
    return collect


ExtractList(out_list, 'energies_pot')

In [ ]:
macro_node.outputs

In [ ]:
macro_graph.edges

In [ ]:
(macro_graph.nodes["Potential"].node.inputs.structure.value.node.label), (macro_graph.nodes["Potential"].node.inputs.structure.value.value.node.label)

In [ ]:
(macro_graph.nodes["InitLammps"].node.inputs.calculator.value.node.label), (macro_graph.nodes["InitLammps"].node.inputs.calculator.value.value.node.label)

In [ ]:
# connect inner node input directly with outer node, eliminate in execution macro input port
for graph_node in macro_graph.nodes.values():
    values = graph_node.node.inputs.data['value']
    labels = graph_node.node.inputs.data['label']
    for port_label, port_value in zip(labels, values):
        # print('label: ', port_label)
        if isinstance(port_value, (base.Port)):
            # print(port_label, type(port_value.value))
            if isinstance(port_value.value, (base.Port)):
                print('double: ', port_value.value.label, port_value.value.node.label)
                graph_node.node.inputs.__setattr__(port_label, port_value.value) 

# macro_graph.nodes['Potential'].node.inputs.structure.value.value
pf.graph.nodes["Bulk"].node.run()
pf.graph.nodes["Potential"].node.run()
base.run_macro_node(macro_node)

In [ ]:
macro_graph.nodes["ListPotentials"].node.inputs.structure.value.value.value

In [ ]:
macro_node.inputs.structure.value.value

In [ ]:
pf.graph.nodes['CubicBulkCell'].node.inputs

In [ ]:
pf.graph.nodes['CubicBulkCell'].node.node_type

In [ ]:
def run_macro_node(macro_node):
    macro_graph = base.get_graph_from_macro_node(macro_node)
    
    output_nodes = list()
    for edge in macro_graph.edges:
        if f'va_o_{macro_node.label}__' in edge.target:
            output_nodes.append(edge.source)
    
    outputs = list() 
    for out_label in set(output_nodes):
        print(f'output node {out_label} of macro {macro_node.label}')
        outputs.append(base.pull_node(macro_graph, out_label)) # use graph theory to avoid recalculating nodes (or use ready)

    if len(outputs) == 1:
        return outputs[0]
    else:
        return set(outputs)

macro_node = pf.graph.nodes['CubicBulkCell'].node
run_macro_node(macro_node)

In [ ]:
pf.graph.nodes['CubicBulkCell'].node.run()

In [ ]:
pf.graph.nodes['bulk'].node.inputs.name.value.value  # input port of macro

In [ ]:
pf.graph.nodes['bulk'].node.run()

In [ ]:
pf.graph.nodes['Plot3d'].node.inputs.structure.value.value #.name.value.value

In [ ]:
base._graph_to_gui(sub_graph)

In [ ]:
def Subgraph(structure, ):

    from pyiron_workflow import Workflow
    import pyiron_nodes

    wf = Workflow('subgraph')

    wf.Repeat = pyiron_nodes.atomistic.structure.transform.Repeat(structure=structure) 
    wf.Repeat_2 = pyiron_nodes.atomistic.structure.transform.Repeat(structure=wf.Repeat) 
    wf.Plot3d = pyiron_nodes.atomistic.structure.view.Plot3d(structure=structure) 

    return wf.Repeat_2

structure = pn.atomistic.structure.build.Bulk("Al", cubic=True).run()
Subgraph(structure=structure).pull()

In [ ]:
def my_func(structure=None, repeat_scalar=1):
    from pyiron_workflow import Workflow
    import pyiron_nodes

    wf = Workflow("subgraph")

    # wf.va_i_subgraph__structure = __main__.InputNode()
    wf.Repeat = pyiron_nodes.atomistic.structure.transform.Repeat(structure=structure, repeat_scalar=repeat_scalar)
    wf.Repeat_2 = pyiron_nodes.atomistic.structure.transform.Repeat(structure=wf.Repeat)
    wf.Plot3d = pyiron_nodes.atomistic.structure.view.Plot3d(structure=structure)

    return wf.Repeat_2.pull()
   
structure = pn.atomistic.structure.build.Bulk("Al", cubic=True).run()
my_func(structure=structure, repeat_scalar=2);


In [ ]:
structure = pn.atomistic.structure.build.Bulk("Al", cubic=True).run()
import pyiron_nodes

wf = Workflow("subgraph")

# wf.va_i_subgraph__structure = __main__.InputNode()
wf.Repeat = pyiron_nodes.atomistic.structure.transform.Repeat(structure=structure)
wf.Repeat_2 = pyiron_nodes.atomistic.structure.transform.Repeat(structure=wf.Repeat, repeat_scalar=1)
wf.Plot3d = pyiron_nodes.atomistic.structure.view.Plot3d(structure=structure)

wf.run()


## From Graph

In [ ]:
import pyiron_nodes as pn


@as_function_node
def InputNode(structure=None):
    return structure


@as_function_node
def OutputNode(structure=None):
    return structure

InputStructure = "va_i_subgraph__structure"
OutputStructure = "va_o_subgraph__structure"

sub_graph = base.Graph(label="subgraph")
sub_graph += InputNode(label=InputStructure)
#sub_graph += pn.atomistic.structure.transform.Repeat(label="Repeat")
#sub_graph += pn.atomistic.structure.transform.Repeat(label="Repeat_2")
#sub_graph += pn.atomistic.structure.view.Plot3d(label="Plot3d")
sub_graph += OutputNode(label=OutputStructure)

#sub_graph += base.GraphEdge("Repeat", "Repeat_2", "structure", "structure")
#sub_graph += base.GraphEdge(InputStructure, "Repeat", "structure", "structure")
#sub_graph += base.GraphEdge(InputStructure, "Plot3d", "structure", "structure")
#sub_graph += base.GraphEdge("Repeat_2", OutputStructure, "structure", "structure")

graph = base.Graph(label="Workflow")
graph += pn.atomistic.structure.build.Bulk(name="Al", label="structure")
graph += pn.atomistic.structure.transform.Repeat(label="repeat")
graph += sub_graph
graph.nodes["subgraph"].expanded = False
graph.nodes["subgraph"].node = base.graph_to_node(sub_graph)

graph += pn.atomistic.structure.view.Plot3d(label="plot")

graph += base.GraphEdge("structure", "repeat", "structure", "structure")
graph += base.GraphEdge(OutputStructure, "plot", "structure", "structure")
graph += base.GraphEdge("repeat", InputStructure, "structure", "structure")
# graph += base.GraphEdge("subgraph", "plot", "structure", "structure")
# graph += base.GraphEdge("repeat", "subgraph", "structure", "structure")

graph = base.remove_node(graph, "Repeat")
graph = base.remove_node(graph, "Repeat_2")


base.GuiGraph(graph);

In [ ]:
sub_graph = base.Graph(label="subgraph")
sub_graph += pn.atomistic.structure.build.Bulk(label='Bulk', name='Al')
sub_graph += pn.atomistic.structure.transform.Repeat(label="Repeat", repeat_scalar=2)
sub_graph += pn.atomistic.structure.transform.Repeat(label="Repeat_2")
sub_graph += pn.atomistic.structure.view.Plot3d(label="Plot3d")
# sub_graph += OutputNode(label=OutputStructure)

sub_graph += base.GraphEdge("Repeat", "Repeat_2", "structure", "structure")
sub_graph += base.GraphEdge("Bulk", "Repeat", "structure", "structure")
sub_graph += base.GraphEdge("Bulk", "Plot3d", "structure", "structure")
# sub_graph += base.GraphEdge("Repeat_2", OutputStructure, "structure", "structure")

# sub_graph = base.update_execution_graph(sub_graph)

graph = base.Graph(label="Workflow")
graph += pn.atomistic.structure.build.Bulk(name="Al", label="structure")
graph += pn.atomistic.structure.transform.Repeat(label="repeat")
# graph += base.graph_to_node(sub_graph)
graph += sub_graph 
graph.nodes["subgraph"].expanded = False
graph.nodes["subgraph"].node = base.graph_to_node(sub_graph)

graph += pn.atomistic.structure.view.Plot3d(label="plot")

graph += base.GraphEdge("structure", "repeat", "structure", "structure")
# graph += base.GraphEdge(OutputStructure, "plot", "structure", "structure")
# graph += base.GraphEdge("repeat", InputStructure, "structure", "structure")

base.get_inputs_of_graph(sub_graph, exclude_unconnected_default_ports=True)
base.GuiGraph(graph)

In [ ]:
node = base.graph_to_node(sub_graph)
node.inputs.repeat_scalar = 1
node.inputs

node.run()

In [ ]:
graph.nodes["subgraph"].expanded = True
graph.nodes

In [ ]:
xx

In [ ]:
print(graph.nodes["subgraph"].expanded)
# Collapse a node
collapsed_graph = base.collapse_node(graph, "subgraph")

# Expand a node
expanded_graph = base._expand_node(graph, "subgraph")
print(graph.nodes["subgraph"].expanded)


In [ ]:
graph.label

In [ ]:
pf = gui.PyironFlow([graph])
pf.gui

In [ ]:
base.GuiGraph(collapsed_graph);

In [ ]:
graph.nodes["subgraph"].node.__getstate__() # correct function has to be constructed and stored in the node!

## From workflow

In [ ]:
wf = Workflow('lammps_full')

# wf.structure = pn.atomistic.structure.build.Bulk('Al', cubic=True)
wf.lammps = pn.atomistic.engine.lammps.Code1() # structure=wf.structure)
# wf.md_output = pn.atomistic.calculator.data.OutputCalcMD(dataclass=wf.lammps.outputs.generic)

In [ ]:
graph = base.Graph(label='test')
graph += pn.atomistic.engine.lammps.Code1()
graph.nodes["Code1"].expanded = False

base.GuiGraph(graph)
                   

In [ ]:
# graph.nodes['Code1'].graph

In [ ]:
graph = base.get_full_graph_from_wf(wf)


In [ ]:
base.get_import_path_from_type(pn.atomistic.engine.lammps.Code1)

In [ ]:
# wf = Workflow("flow")
# wf.LoadGraph = pn.graphs.flow.LoadGraph('lammps_full')
# wf.DisplayNodes = pn.graphs.flow.DisplayNodes(graph=wf.LoadGraph)
# wf.DisplayNodes.pull()

# graph = base.get_full_graph_from_wf(wf)

In [ ]:
# base.GuiGraph(pf.graph)

In [ ]:
pf = gui.PyironFlow([wf, 'flow_expand', 'edges', 'iterCellSize'])
pf.gui

In [ ]:
from pyiron_workflow import as_macro_node

In [ ]:
graph = base.Graph('test')
graph += pf.graph.nodes['Repeat']

pf.graph.nodes['Repeat'].node.outputs.structure

In [ ]:
base.get_inputs_of_graph(pf.graph, exclude_unconnected_default_ports=True) 

In [ ]:
base.get_inputs_of_graph(graph, exclude_unconnected_default_ports=False)

In [ ]:
base.get_outputs_of_graph(pf.graph)

In [ ]:
print(base.graph_to_code(pf.graph))

In [ ]:
node = base.graph_to_node(pf.graph, exclude_unconnected_default_ports=False)
node.inputs.repeat_scalar = 1
node.run()
node.inputs

In [ ]:
import inspect

inspect.getsource(node._func)

In [ ]:
import types
from functools import partial

def graph_to_code(graph):
    graph = base.get_updated_graph(graph)
    graph = base.topological_sort(graph)
    graph = base.get_code_from_graph(graph)
    return graph


def graph_to_node(graph: base.Graph, exclude_unconnected_default_ports=True) -> base.Node:
    
    function_string = graph_to_code(graph)
    
    # Create a dictionary to serve as the local namespace
    virtual_namespace = {}
    
    # Execute the function string in the local namespace
    exec(function_string, globals(), virtual_namespace)
    
    # Retrieve the function from the local namespace
    func = virtual_namespace[graph.label]

    node = base.Node(
        func=func,
        label=graph.label,
        node_type="graph",
        inputs=base.get_inputs_of_graph(graph, exclude_unconnected_default_ports=True),
        outputs=base.get_outputs_of_graph(graph),
    )
    node.label = graph.label  # should not be necessary


    def _run(node):
        port = func(**node.kwargs)

        return port.node._workflow.run()
    
    node._run = types.MethodType(_run, node)

    return node



node = graph_to_node(pf.graph)
node.inputs.repeat_scalar = 1

node.run()
node.outputs.structure.value

In [ ]:


function_string = graph_to_code(pf.graph)

# Create a dictionary to serve as the local namespace
virtual_namespace = {}

# Execute the function string in the local namespace
exec(function_string, globals(), virtual_namespace)

# Retrieve the function from the local namespace
func = virtual_namespace['Workflow_3']

port = func(name='Fe', repeat_scalar=2)
port.node._workflow.run()

In [ ]:
# Define the function as a string
function_string = """
def Workflow_3(name: str, cubic: bool = False, repeat_scalar: int = 1):

    from pyiron_workflow import Workflow
    import pyiron_nodes

    wf = Workflow('Workflow_3')

    wf.Bulk = pyiron_nodes.atomistic.structure.build.Bulk(name=name, cubic=cubic) 
    wf.Repeat = pyiron_nodes.atomistic.structure.transform.Repeat(structure=wf.Bulk, repeat_scalar=repeat_scalar) 

    return wf.Repeat.outputs.structure
"""

# Create a dictionary to serve as the local namespace
virtual_namespace = {}

# Execute the function string in the local namespace
exec(function_string, globals(), virtual_namespace)

# Retrieve the function from the local namespace
func = virtual_namespace['Workflow_3']

port = func(name='Fe', repeat_scalar=2)
port.node._workflow.run()

In [ ]:
port.label

In [ ]:
virtual_namespace

In [ ]:
xx

In [ ]:
label = 'lammps'
print(graph.nodes[label].expanded)
# Collapse a node
collapsed_graph = base.collapse_node(graph, label)
print(graph.nodes[label].expanded)

# Expand a node
expanded_graph = base.expand_node(graph, label)
print(graph.nodes[label].expanded)

In [ ]:
collapsed_graph.edges

In [ ]:
base.GuiGraph(collapsed_graph)

In [ ]:
base.GuiGraph(expanded_graph)

In [ ]:
base.GuiGraph(graph)

In [ ]:
graph.nodes

In [ ]:
collapsed_graph.nodes

In [ ]:
graph.nodes['lammps'].expanded = False

u_graph = base.get_updated_graph(graph)

graph.nodes['lammps'].expanded 



In [ ]:
base.GuiGraph(u_graph)